In [ ]:

from json import *
import numpy as np
with open('facebook.json','r')as f:
    facebook=f.read()
facebook=JSONDecoder().decode(facebook)

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9999
set_session(tf.Session(config=config))

In [ ]:
word_dict={'PADDING':[0,99999],'UNK':[1,99999]}

pos_dict={'PADDING':0,'UNK':1}
for i in facebook:
    tp=i
    for word in tp[0]:
        if not word in word_dict:
            word_dict[str(word)]=[len(word_dict),1]
        else:
            word_dict[str(word)][1]+=1
    for pos in tp[1]:
        if not pos in pos_dict:
            pos_dict[str(pos)]=len(pos_dict)        



In [ ]:
print(len(word_dict))
print(len(pos_dict))

In [ ]:
maxlen=100
count=0
all_data=[]
all_pos=[]
all_label=[]

for senter in facebook:
    sent=[]
    pos=[]
    

    for word in senter[0]:
        if word_dict[str(word)][1]>=1:
            sent.append(word_dict[str(word)][0])
    for poser in senter[1]:
        pos.append(pos_dict[str(poser)])
    #labelk.append([float(senter[2][0]),float(senter[2][1])])
    if len(sent)<=100:
        sent=sent+[0]*(100-len(sent))
        pos=pos+[0]*(100-len(pos))
        all_data.append(sent)
        all_pos.append(pos)
        all_label.append([float(senter[2][0]),float(senter[2][1])])

In [ ]:
embdict=dict()
plo=0
line=0
with open('/home/wuch/GoogleNews-vectors-negative300.bin','rb')as f:
    header = f.readline()
    vocab_size, layer1_size = map(int, header.split())
    binary_len = np.dtype('float32').itemsize * layer1_size
    for line in range(vocab_size):
        word = []
        while True:
            ch = f.read(1).decode("utf-8", 'ignore')
            if ch ==' ':
                word = ''.join(word)
                break
            if ch != '\n':
                word.append(ch)
        if len(word) != 0:
            tp= np.fromstring(f.read(binary_len), dtype='float32')
            if word in word_dict:
                embdict[word]=tp.tolist()
                if plo%100==0:
                    print(plo,line,word)
                plo+=1
                #print(word,tp)
        else:
            f.read(binary_len)


In [ ]:

print(len(word_dict))
lister=[0]*len(word_dict)
xp=np.zeros(300,dtype='float32')

for i in embdict.keys():
    lister[word_dict[i][0]]=np.array(embdict[i],dtype='float32')

    
norm=np.random.uniform(-0.1,0.1,(len(word_dict),300))

for i in range(len(lister)):
    if type(lister[i])==int:
        lister[i]=np.reshape(norm[i], 300)
lister[0]=np.zeros(300,dtype='float32')
lister=np.array(lister,dtype='float32')
print(lister.shape)



In [ ]:

import random
l1=list(range(2790))
random.shuffle(l1)

all_data0=[]
all_pos0=[]
all_label0=[]

for x in l1:
    all_pos0.append(all_pos[x])
    all_label0.append(all_label[x])


In [ ]:

count=0
all_data000=[]
for x in l1:
    
    count+=1
    print(count)    
    tt1=[]       
    for xx in all_data[x]:
        tp=[0]*xx+[1]+[0]*(8589-xx)
        tt1.append(tp)
    all_data000.append(tt1)

In [ ]:
for x in l1:
    all_data0.append(all_data[x])

In [ ]:
for fold in range(10):
    all_index=range(len(all_data0))
    indexer=random.sample(all_index,int(len(all_data0)*0.9))
    
    labeled_ratio=0.2
    zero_mask=[[1]]*(int(len(all_data0)*0.9)*labeled_ratio)+[[0]]*(int(len(all_data0)*0.9)*(1-labeled_ratio))
    
    test_index=list(set(all_index)-set(indexer))
    
    train_item=np.array(all_data0,dtype='int32')[indexer]
    train_pos=np.array(all_pos0,dtype='int32')[indexer]
    train_label=np.array(all_label0,dtype='float32')[indexer]
    
    test_item=np.array(all_data0,dtype='int32')[test_index]
    test_pos=np.array(all_pos0,dtype='int32')[test_index]
    test_label=np.array(all_label0,dtype='float32')[test_index]
    from keras.layers import GRU,Bidirectional
    import keras.backend as K
    from keras.models import Sequential,Model
    from keras.layers import Dense, Dropout, Input,Activation,wrappers,Merge,merge,core,advanced_activations,MaxPooling1D,GlobalAveragePooling1D
    from keras.layers import LSTM,Conv1D,GlobalMaxPooling1D,SimpleRNN
    from keras.layers import Masking, Embedding,Flatten,RepeatVector,Permute,TimeDistributed
    from keras.regularizers import l2
    from keras.callbacks import ModelCheckpoint,EarlyStopping
    import tensorflow as tf
    import random
    sequenceLength =100
    fmin=0
    x_train = Input(batch_shape=(None,sequenceLength),dtype='int32')
    x_train2 = Input(batch_shape=(None,sequenceLength),dtype='int32')
    y_train = Input(batch_shape=(None,3),dtype='float32')
    
    
    x_test = Input(batch_shape=(None,sequenceLength),dtype='int32')
    x_test2 = Input(batch_shape=(None,sequenceLength),dtype='int32')
    y_test = Input(batch_shape=(None,3),dtype='float32')
    
    emb_drop=0.5
    pos_emb_drop=0.5
    
    global_drop=0.5
    
    hid1=100
    emb = Embedding(len(word_dict), 300,weights=[lister],input_length=sequenceLength,mask_zero=False)(x_train)
    drop_out_emb = Dropout(emb_drop)(emb)
    
    pos_emb = Embedding(47, 50,input_length=sequenceLength,mask_zero=False,dropout=pos_emb_drop)(x_train2)
    drop_out_pos_emb = Dropout(pos_emb_drop)(pos_emb)
    
    merged = merge([drop_out_emb, drop_out_pos_emb], mode='concat',concat_axis=2)
    

    
    flat_beta=Bidirectional(LSTM(hid1,return_sequences=True))(merged)
    flat_beta2=Bidirectional(LSTM(hid1,return_sequences=False))(flat_beta)
    flat_betad = Dropout(global_drop)(flat_beta2)
    
    outer=Dense(3)(flat_betad)

 
    h =LSTM(100,batch_input_shape=(batch_size,sequenceLength, 350),return_sequences=False,dropout_W=0.5,dropout_U=0.5)(merged)

    h_merge=merge([h, outer], mode='concat',concat_axis=-1)
    z_mean = Dense(latent_dim)(h_merge)
    z_log_sigma = Dense(latent_dim)(h_merge)
    
    
    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                                  stddev=epsilon_std)
        return z_mean + K.exp(z_log_sigma / 2) * epsilon
    
    z = Lambda(sampling)([z_mean, z_log_sigma])
    
    
    decoder = RecurrentSequential(decode=True, output_length=sequenceLength,unroll=False, stateful=False)
    decoder.add(Dropout(0.5))
    decoder.add(LSTMDecoderCell(output_dim=100, hidden_dim=100))
    h_decoded = decoder(z)
    
    x_decoded_mean = TimeDistributed(Dense(len(word_dict),activation='softmax'))(h_decoded)

    model = Model(input=[x_train2,xx,x_train3], output=[outer,x_decoded_mean])
    def vae_loss(x, x_decoded_mean):

        xent_loss = K.mean(metrics.categorical_crossentropy(x, x_decoded_mean))
        kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
        return xent_loss+kl_loss

    def myloss(y_true, y_pred):
        return K.mean(x_train3*K.abs(y_pred - y_true), axis=-1)
    

    model.compile(loss=[myloss,vae_loss],loss_weights=[0.5*(1/(labeled_ratio)), 1.],
                  optimizer='rmsprop',metrics=['mae'])
    early=EarlyStopping(monitor='val_loss',patience=10)

    model.fit([train_pos,train_item0,train_mask],[train_label,train_item0], batch_size=batch_size, nb_epoch=50,shuffle=True,validation_split=0.1,callbacks=[early])
        
    classes=model.predict([test_item,test_pos], batch_size=64)
    
    from scipy.stats.stats import pearsonr
    mae=np.mean(np.abs(classes-test_label),axis=0)
    rmse=np.sqrt(np.mean(np.square(classes-test_label),axis=0))
    pcc1=pearsonr(classes[:,0],test_label[:,0])[0]
    pcc2=pearsonr(classes[:,1],test_label[:,1])[0]
    pcc3=pearsonr(classes[:,2],test_label[:,2])[0]

    print('mae:',mae[0],mae[1],mae[2])
    print('pcc:',pcc1,pcc2,pcc3)